In [61]:
from fastai.vision.all import *
import matplotlib.pyplot as plt
import numpy as np



In [62]:
# Charger le jeu de données MNIST
path = untar_data(URLs.MNIST)

# Créer des DataLoaders pour l'entraînement et la validation
dls = ImageDataLoaders.from_folder(
    path,
    train='training',
    valid='testing',
    seed=42,
    bs=64,
    item_tfms=Resize(28)  # MNIST images are 28x28
)



In [63]:
# Explorer la structure des données
print(f"Catégories: {dls.vocab}")
print(f"Nombre de batches d'entraînement: {len(dls.train)}")
print(f"Nombre de batches de validation: {len(dls.valid)}")

# Fonction pour afficher une image MNIST
def display_image(img, label, index):
    plt.figure(figsize=(4, 4))
    plt.imshow(img, cmap='gray')
    plt.title(f"Chiffre: {label}")
    plt.axis('off')
    plt.savefig(f'mnist_example_{index}.png')
    plt.close()



Catégories: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
Nombre de batches d'entraînement: 937
Nombre de batches de validation: 157


In [64]:
# Afficher les 5 premières images du jeu d'entraînement
batch = dls.train.one_batch()
images, labels = batch[0][:5], batch[1][:5]
for i in range(5):
    img = images[i].permute(1, 2, 0).cpu().numpy()  # Convertir tensor en numpy
    label = dls.vocab[labels[i]]
    display_image(img, label, i)



In [65]:
# Prétraitement : normalisation des pixels
def normalize_images(batch):
    # Normaliser (0-255 -> 0-1)
    batch[0] = batch[0] / 255.0
    return batch

# Appliquer la normalisation via un transform
dls.after_batch = normalize_images



In [66]:
# Vérifier les valeurs normalisées
sample_batch = dls.train.one_batch()
sample_image = sample_batch[0][0].cpu().numpy()
print(f"Valeurs min/max après normalisation: {sample_image.min():.2f}/{sample_image.max():.2f}")

# Exemple de filtrage : sélectionner uniquement les images du chiffre 5
def filter_fives(items):
    return [item for item in items if '5' in str(item[1])]



Valeurs min/max après normalisation: 0.00/1.00


In [67]:
# Créer un nouveau DataLoader avec uniquement le chiffre 5
filtered_dls = ImageDataLoaders.from_folder(
    path,
    train='training',
    valid='testing',
    seed=42,
    bs=64,
    item_tfms=Resize(28),
    get_items=filter_fives
)

print(f"Nombre d'images du chiffre 5 (entraînement): {len(filtered_dls.train_ds)}")


Nombre d'images du chiffre 5 (entraînement): 60000
